[View in Colaboratory](https://colab.research.google.com/github/Stallon-niranjan/Inkers_EIP_WDocs/blob/master/NIRANJAN_ANANDAKUMAR_BATCH_2_ASSIGNMENT4B.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 200
num_classes = 10
epochs = 150
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 25s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/150
50000/50000 [==============================] - 144s 3ms/step - loss: 1.7573 - acc: 0.3404 - val_loss: 2.1865 - val_acc: 0.3219
Epoch 2/150
44000/50000 [=========================>....] - ETA: 14s - loss: 1.3960 - acc: 0.4866

50000/50000 [==============================] - 131s 3ms/step - loss: 1.3871 - acc: 0.4889 - val_loss: 1.5146 - val_acc: 0.4822
Epoch 3/150
50000/50000 [==============================] - 131s 3ms/step - loss: 1.2381 - acc: 0.5475 - val_loss: 1.5219 - val_acc: 0.4869
Epoch 4/150
22600/50000 [============>.................] - ETA: 1:07 - loss: 1.1770 - acc: 0.5707

50000/50000 [==============================] - 131s 3ms/step - loss: 1.1573 - acc: 0.5782 - val_loss: 1.2615 - val_acc: 0.5668
Epoch 5/150
50000/50000 [==============================] - 131s 3ms/step - loss: 1.1023 - acc: 0.6007 - val_loss: 1.1833 - val_acc: 0.5903
Epoch 6/150
17600/50000 [=========>....................] - ETA: 1:20 - loss: 1.0648 - acc: 0.6077

50000/50000 [==============================] - 131s 3ms/step - loss: 1.0480 - acc: 0.6181 - val_loss: 1.0847 - val_acc: 0.6166
Epoch 7/150
50000/50000 [==============================] - 131s 3ms/step - loss: 1.0091 - acc: 0.6335 - val_loss: 1.1991 - val_acc: 0.5875
Epoch 8/150
16400/50000 [========>.....................] - ETA: 1:23 - loss: 0.9842 - acc: 0.6431

50000/50000 [==============================] - 131s 3ms/step - loss: 0.9755 - acc: 0.6492 - val_loss: 1.2182 - val_acc: 0.5883
Epoch 9/150
50000/50000 [==============================] - 131s 3ms/step - loss: 0.9403 - acc: 0.6609 - val_loss: 1.8296 - val_acc: 0.5092
Epoch 10/150
16000/50000 [========>.....................] - ETA: 1:24 - loss: 0.9266 - acc: 0.6654

...

50000/50000 [==============================] - 131s 3ms/step - loss: 0.3745 - acc: 0.8680 - val_loss: 0.5618 - val_acc: 0.8342
Epoch 143/150
50000/50000 [==============================] - 132s 3ms/step - loss: 0.3721 - acc: 0.8682 - val_loss: 0.6382 - val_acc: 0.8183
Epoch 144/150
15400/50000 [========>.....................] - ETA: 1:26 - loss: 0.3605 - acc: 0.8729

50000/50000 [==============================] - 132s 3ms/step - loss: 0.3730 - acc: 0.8684 - val_loss: 0.6497 - val_acc: 0.8094
Epoch 145/150
50000/50000 [==============================] - 132s 3ms/step - loss: 0.3712 - acc: 0.8691 - val_loss: 0.5727 - val_acc: 0.8303
Epoch 146/150
15400/50000 [========>.....................] - ETA: 1:26 - loss: 0.3550 - acc: 0.8781

50000/50000 [==============================] - 132s 3ms/step - loss: 0.3674 - acc: 0.8710 - val_loss: 0.5601 - val_acc: 0.8299
Epoch 147/150
50000/50000 [==============================] - 132s 3ms/step - loss: 0.3713 - acc: 0.8675 - val_loss: 0.6903 - val_acc: 0.8043
Epoch 148/150
15400/50000 [========>.....................] - ETA: 1:25 - loss: 0.3655 - acc: 0.8753

50000/50000 [==============================] - 132s 3ms/step - loss: 0.3683 - acc: 0.8708 - val_loss: 0.6753 - val_acc: 0.8074
Epoch 149/150
50000/50000 [==============================] - 132s 3ms/step - loss: 0.3628 - acc: 0.8702 - val_loss: 0.5743 - val_acc: 0.8289
Epoch 150/150
15400/50000 [========>.....................] - ETA: 1:26 - loss: 0.3624 - acc: 0.8722

50000/50000 [==============================] - 132s 3ms/step - loss: 0.3677 - acc: 0.8704 - val_loss: 0.6777 - val_acc: 0.8115


In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 14s 1ms/step
Test loss: 0.6777336399555206
Test accuracy: 0.8115


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')